In [13]:
from datasets import load_dataset
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import sentencepiece as spm
from torch.nn.utils.rnn import pad_sequence

In [14]:
data = load_dataset("spider")

Found cached dataset spider (C:/Users/18327/.cache/huggingface/datasets/spider/spider/1.0.0/4e5143d825a3895451569c8b9b55432b91a4bc2d04d390376c950837f4680daa)


  0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
# Data Sample
pd.DataFrame(data["train"]).head()

,db_id,query,question,query_toks,query_toks_no_value,question_toks
0,department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...","[How, many, heads, of, the, departments, are, ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","[List, the, name, ,, born, state, and, age, of..."
2,department_management,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","[List, the, creation, year, ,, name, and, budg..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...,"[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...","[What, are, the, maximum, and, minimum, budget..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...","[What, is, the, average, number, of, employees..."


In [16]:
train_df = pd.DataFrame(data["train"])[["query", "question"]]
val_df = pd.DataFrame(data["validation"])[["query", "question"]]

In [17]:
vocab_size = 5000
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 350 # what is the context?
max_iteration = 100
eval_interval = 1
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0

In [18]:
device

'cuda'

In [19]:
# pd.concat([question,query]).to_csv(r'sptrain.txt', header=None, index=None, sep=' ', mode='w')
# spm.SentencePieceTrainer.train(f'--input=sptrain.txt --model_prefix=m --vocab_size={vocab_size}')

# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

encode = lambda s: sp.encode(s, out_type=int, enable_sampling=True, alpha=0.1, nbest_size=-1,)
decode = lambda l: sp.decode(l)

In [20]:
def get_batch(split):
    data =  train_df if split == "train" else val_df
    data = list(zip(data["question"],data["query"]))
       
    indices = torch.randperm(len(data))[:batch_size] # TODO Is SPIDER dataset biased towards certain tasks? Then this is good way to randomize.
    
    # Encode data in random batches.
    x = [torch.tensor(encode(data[i][0]), dtype=torch.long) for i in indices]
    y = [torch.tensor(encode(data[i][1]), dtype=torch.long) for i in indices]
    
    # Find max length between both x and y.
    # max_len_x = max(len(row) for row in x)
    # max_len_y = max(len(row) for row in y)
    # max_len = max(max_len_x, max_len_y)
    max_len = block_size
    
    # Use max length to equally pad zeros to both variables.
    x = [torch.cat([row, torch.zeros(max_len - len(row), dtype=torch.long)]) for row in x]
    y = [torch.cat([row, torch.zeros(max_len - len(row), dtype=torch.long)]) for row in y]
    
    # Include batch as the first dimension.
    x = pad_sequence(x, batch_first=True)
    y = pad_sequence(y, batch_first=True)
    
    x, y = x.to(device), y.to(device)
    
    return x, y

In [21]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [24]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [25]:
# super simple bigram model
class SQLTModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx



In [26]:
model_state_dict = torch.load('model.pt')
model = SQLTModel()
model.load_state_dict(model_state_dict)

m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.866696 M parameters


In [27]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [34]:
max_iteration = 20
for iter in range(max_iteration):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iteration - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
torch.save(m.state_dict(), 'model.pt')

step 0: train loss 0.9726, val loss 1.0723
step 1: train loss 1.0089, val loss 1.0811
step 2: train loss 1.0191, val loss 1.0826
step 3: train loss 0.9931, val loss 1.0651
step 4: train loss 1.0136, val loss 1.0688
step 5: train loss 0.9856, val loss 1.0678
step 6: train loss 0.9778, val loss 1.0638
step 7: train loss 0.9927, val loss 1.0617
step 8: train loss 0.9820, val loss 1.0570
step 9: train loss 0.9833, val loss 1.0491
step 10: train loss 0.9856, val loss 1.0756
step 11: train loss 0.9687, val loss 1.0535
step 12: train loss 0.9840, val loss 1.0542
step 13: train loss 0.9847, val loss 1.0523
step 14: train loss 0.9790, val loss 1.0444
step 15: train loss 0.9694, val loss 1.0653
step 16: train loss 0.9730, val loss 1.0753
step 17: train loss 0.9962, val loss 1.0528
step 18: train loss 0.9816, val loss 1.0535
step 19: train loss 0.9893, val loss 1.0643


In [160]:
torch.save(m.state_dict(), 'model.pt')

In [28]:
context = torch.tensor([[   3, 1855,   21, 3831,  122,    7,   44,   36,  303,    3,   21, 2463,
        1879,    3,  161,    3,   21, 1610,    3,   62,   21,  381,  302,  302,
           3,  679,    3,   21, 1610, 4881,  350,  144,  712,  170]], dtype=torch.long, device=device)
decode(context.squeeze().tolist())

'Return the name and job title of the staff with the latest date assigned.'

In [29]:
context = encode("What is the headcount of human resources department?")
context = torch.tensor([context], dtype=torch.long, device=device)

In [35]:
# generate from the model
output = m.generate(context, max_new_tokens=100)
print(decode(output.squeeze().tolist()))

What is the headcount of human resources department? ⁇ p ⁇  client B ORDER J ⁇ i ⁇ T ER RO AS2 F AS YEARY A RussiayOphysicianN,LgJ ⁇  ⁇ _entLeYo= match. ⁇  ⁇ code  T=scholarshipOI TidON_Tdee( count urNt SHE cr_tee , F ⁇  JuniorSS c2ID ⁇  ⁇ H ⁇ 2CL ⁇  ⁇  ⁇ r BS ⁇  ⁇  ⁇  ⁇ =AL


In [177]:
output

tensor([[   3,  272, 1423,   74,    3,  519,   62,   31, 1610,    3, 1974, 1613,
         1029,  122,   21,    3,  161, 2006, 2768,  381,  122, 3681,  226, 2046,
         4546, 3136,  369, 4037,   21, 4463,   28,   18,   38,  547,    0,  107,
            3,  180,   21,   17,   24,    5, 4051,    0,    3,   35,   30,   14,
            3, 1029,    5,   87,    0,    0,   13,    0,  600,   92,   22,   19,
            4,    0,    0,    5,    0,    3,   26,    0,    0,    3,    5,  293,
           30,    0,    4,  381,  340, 1577,    0,    0, 4940,    3,   26,  411,
            0,    4,   34,  111,    3,    0,    0,    0,  150,   34,   25,  519,
          111,   18,    0,   20,    0,    0,  146,    0,    0,    3,   17,    4,
           20,    9,    0,    3,    3,    4,    0,    5,  107,    3,   12,  411,
           10,   29,    3,  122,    0,   15,  676,  129,    4,   87,    0]],
       device='cuda:0')